In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from time import sleep

In [3]:
from importlib import reload

In [4]:
import drivers, lib
from time import sleep
import pickle

In [5]:
from lib import data_management as dm

In [6]:
# reload(drivers.IQAWG)

In [7]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
from drivers.znb import *
# reload(drivers.KeysightAWG)
# from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.Tektronix_AWG5014 import *
from drivers.IQAWG import *
from drivers.E8257D import MXG, EXG

In [8]:
exa = Agilent_EXA_N9010A("EXA")
# ro_awg = KeysightAWG("AWG3")
# q_awg = KeysightAWG("AWG2")
# ro_lo = EXG("PSG")

In [24]:
# awg._visainstrument.close()
awg = Tektronix_AWG5014("TEK1")
ro_awg = IQAWG(AWGChannel(awg, 3), AWGChannel(awg, 4))
q_awg = IQAWG(AWGChannel(awg, 1), AWGChannel(awg, 2))

In [11]:
q_lo = EXG("PSG")

In [ ]:
ro_lo = Agilent_PNA_L("PNA-L2")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_span(0)
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [25]:
ro_lo = Znb("ZNB")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [26]:
# reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [23]:
ro_resonator_frequency = 8.09e9
if_offset = 0e6

In [28]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "CHGRO", -10)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=-9,
                    ssb_power=-50, waveform_resolution=1, iterations=3, minimize_iterlimit=100,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
#                     initial_guess = {'dc_offsets': array([ 0.1,  .1]),
#                                     'dc_offsets_open': array([ 1  ,  1]),
#                                     'if_amplitudes': None,
#                                     'if_offsets': None,
#                                     'if_phase': None})
print(res)

Calibration data for mixer CHGRO
Mixer parameters: {'mixer_id': 'CHGRO', 'iq_attenuation': -10}
Radiation parameters: {'lo_frequency': 8090000000.0, 'lo_power': -9, 'if_frequency': 0.0, 'ssb_power': -50, 'waveform_resolution': 1}
Optimization results: {'dc': -119.59307861328125, 'dc_open': [-49.988399505615234]}
Optimization parameters {'dc_offsets': array([ 0.02092428, -0.0576303 ]), 'dc_offsets_open': array([ 1.51238006,  1.51240882]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 1 m 15.45 s
Finished at: 2017-08-08 14:25:44.464957


In [29]:
dm.save_IQMX_calibration(res)

In [7]:
calibrations=dm.load_IQMX_calibration_database("TEST", -20)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

## Qubit

In [14]:
qubit_frequency = 8.925e9
if_frequency = 50e6

In [19]:
cal = IQCalibrator(q_awg, exa, q_lo, "CHGQ", -10)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=14,
                    ssb_power=-25, waveform_resolution=1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,
                    initial_guess=res.get_optimization_results()[0])
print(res)

Calibration data for mixer CHGQ
Mixer parameters: {'mixer_id': 'CHGQ', 'iq_attenuation': -10}
Radiation parameters: {'lo_frequency': 8975000000.0, 'lo_power': 14, 'if_frequency': 50000000.0, 'ssb_power': -25, 'waveform_resolution': 1}
Optimization results: {'dc': -42.405181884765625, 'if': [-24.797910690307617, -42.16930389404297, -102.828369140625]}
Optimization parameters {'dc_offsets': array([ 0.00028481, -0.05141884]), 'dc_offsets_open': None, 'if_offsets': array([ 0.0002918, -0.0504467]), 'if_amplitudes': array([ 0.23176318,  0.21759294]), 'if_phase': array([-0.17240879])}
Optimization time: 0 h 1 m 39.37 s
Finished at: 2017-08-06 10:10:01.657216


In [32]:
(2.9694)/pi

0.945189376034148

In [18]:
res._dc_offsets = res._if_offsets

In [16]:
res._if_phase

array([ 2.97597793])

In [20]:
dm.save_IQMX_calibration(res)

In [30]:
q_lo.visa_instr.close()

In [31]:
awg._visainstrument.close()